In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl

In [3]:
dfRows = pl.read_parquet("./saved_bolillas_gn.parquet")

display(dfRows.head(10))

id,combo,b1,b2,b3,b4,b5,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,shared_values,shared_values_count,score_row,int_unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,bool,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,i64
"""0102040507""","[1, 2, … 7]",1,2,4,5,7,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.3333,0.0333,[0.0333],1,3.4978,4
"""0102040508""","[1, 2, … 8]",1,2,4,5,8,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.25,0.0333,[0.0333],1,3.4145,5
"""0102040509""","[1, 2, … 9]",1,2,4,5,9,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.2,0.0333,[0.0333],1,3.3645,6
"""0102040510""","[1, 2, … 10]",1,2,4,5,10,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.1667,0.0333,[0.0333],1,3.3312,7
"""0102040511""","[1, 2, … 11]",1,2,4,5,11,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.1429,0.0333,[0.0333],1,3.3074,8
"""0102040512""","[1, 2, … 12]",1,2,4,5,12,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.125,0.0333,[0.0333],1,3.2895,9
"""0102040513""","[1, 2, … 13]",1,2,4,5,13,false,0,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.1111,0.0333,[0.0333],1,3.2756,10
"""0102040514""","[1, 2, … 14]",1,2,4,5,14,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.1,0.0333,[0.0333],1,3.2645,11
"""0102040515""","[1, 2, … 15]",1,2,4,5,15,true,16,1.0,0.0323,0.0323,0.0333,0.0333,1.0,0.5,0.5,0.0909,0.0333,[0.0333],1,3.2554,12


In [9]:
dfRows.filter(
  pl.col("id") == "0621272932"
)

id,combo,b1,b2,b3,b4,b5,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,shared_values,shared_values_count,score_row,int_unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,bool,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,i64
"""0621272932""","[6, 21, … 32]",6,21,27,29,32,false,0,0.05,0.0385,0.0833,0.1429,0.1667,0.05,0.04,0.0385,0.0357,0.1667,"[0.0385, 0.1667]",2,0.8123,0


# Get random ticket

In [61]:
dfRows.sample(n=1)  

id,combo,b1,b2,b3,b4,b5,is_rare_combination,reason,score_left_b1,score_left_b2,score_left_b3,score_left_b4,score_left_b5,score_r8_b1,score_r8_b2,score_r8_b3,score_r8_b4,score_r8_b5,shared_values,shared_values_count,score_row,int_unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,bool,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,list[f64],u8,f64,i64
"""0610263233""","[6, 10, … 33]",6,10,26,32,33,false,0,0.1111,0.0385,0.0435,0.125,0.3333,0.1111,0.0417,0.0345,0.0345,0.3333,[0.3333],1,1.2065,27


# ANALYZE

In [5]:
dfRows.group_by(
  'reason'
).agg(pl.count('reason').alias('count'))

reason,count
i64,u32
0,141902
16,68923


In [6]:
def show_b_stats (col: str):
  df_viz = dfRows.group_by(
    col
  ).agg(pl.count(col).alias('count'))

  chart = df_viz.plot.bar(
    x=col,
    y='count',
    color=col,
  )

  display(df_viz)
  display(chart)


In [7]:
show_b_stats('score_left_b2')
show_b_stats('score_r8_b2')


score_left_b2,count
f64,u32
0.0476,7024
0.0769,1130
0.0588,3292
0.125,158
0.0333,24716
…,…
0.0909,580
0.0385,15933
0.0556,4096


alt.Chart(...)

score_r8_b2,count
f64,u32
0.0476,10116
0.0769,11980
0.0333,634
0.0588,12350
0.125,6877
…,…
0.1667,4330
0.0385,4551
0.0417,7149


alt.Chart(...)

In [8]:
show_b_stats('score_left_b4')
show_b_stats('score_r8_b4')

score_left_b4,count
f64,u32
0.0769,12123
0.125,7149
0.0333,595
0.0476,9501
0.0588,12273
…,…
0.0909,10116
0.0385,4330
0.0417,6877


alt.Chart(...)

score_r8_b4,count
f64,u32
0.0476,7164
0.0769,1015
0.0333,26614
0.125,156
0.0588,3285
…,…
0.0909,576
0.0385,15822
0.0556,4057


alt.Chart(...)